In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

**Repository Version** 
This notebook was run at commit:
`21c2fba3eeaefeaf4c07eb48c4c7e2123877f8fb`

# Orthogonality Cross-Check
This notebook will compare the results of fits to GlueX-I data in the orthogonal pairs of diamond orientations, as a first check that fits are behaving as expected. 

## Setup

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import warnings

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/spin-1-orthogonal/float-dsratio/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

The files we'll be analyzing are done in the following manner:
* Varying bins of $-t$
  * [0.1, 0.2], [0.2, 0.3], [0.3, 0.5], [0.5, 1.0]
* 20 MeV width bins of mass in the range $1.0 < M_{\omega\pi^0} < 2.0~\text{GeV}$
* Spin $J=1$ amplitudes only, with a common ratio and phase parameter between all $D$ and $S$ waves
* Remove events with $M_{p\pi^0} < 1.4~\text{GeV}$ to remove baryon contributions

In [ ]:
%%bash
# print out the submission files to view the full list of parameters
cat $STUDY_DIR/0-90/submission_0_90.YAML
cat $STUDY_DIR/45-135/submission_45_135.YAML

In [ ]:
from typing import Dict

# Load in the already preprocessed fit results
all_results_0_90 = {}
all_results_45_135 = {}
all_acc_corrected_results_0_90 = {}
all_acc_corrected_results_45_135 = {}

for bin_dir in sorted(os.listdir(f"{STUDY_DIR}/0-90/")):
    if not os.path.isdir(f"{STUDY_DIR}/0-90/{bin_dir}"):
        continue
    print(f"Loading results from 0-90/{bin_dir}")
    with open(f"{STUDY_DIR}/0-90/{bin_dir}/preprocessed_results.pkl", "rb") as f:
        data = pkl.load(f)
    all_results_0_90[bin_dir] = ResultManager(**data)

    with open(f"{STUDY_DIR}/0-90/{bin_dir}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
    all_acc_corrected_results_0_90[bin_dir] = ResultManager(**data)

for bin_dir in sorted(os.listdir(f"{STUDY_DIR}/45-135/")):
    if not os.path.isdir(f"{STUDY_DIR}/45-135/{bin_dir}"):
        continue
    print(f"Loading results from 45-135/{bin_dir}")
    with open(f"{STUDY_DIR}/45-135/{bin_dir}/preprocessed_results.pkl", "rb") as f:
        data = pkl.load(f)
    all_results_45_135[bin_dir] = ResultManager(**data)

    with open(f"{STUDY_DIR}/45-135/{bin_dir}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
    all_acc_corrected_results_45_135[bin_dir] = ResultManager(**data)

# let interpreter know the types explicitly so we can easily access methods
all_results_0_90: Dict[str, ResultManager]
all_results_45_135: Dict[str, ResultManager]
all_acc_corrected_results_0_90: Dict[str, ResultManager]
all_acc_corrected_results_45_135: Dict[str, ResultManager]

## Analysis

### Standard Plots
First lets take a look at the standard set of plots to diagnose potential problem areas

In [ ]:
for t_bin_0_90, t_bin_45_135 in zip(sorted(pathlib.Path(f"{STUDY_DIR}/0-90/").iterdir()), sorted(pathlib.Path(f"{STUDY_DIR}/45-135/").iterdir())):
    if not t_bin_0_90.is_dir() or not t_bin_45_135.is_dir():
        continue
    utils.big_print(f"t bin: {t_bin_0_90.name}", 2.0)
    for pdf_0_90 in pathlib.Path(t_bin_0_90 / "plots").glob("*.pdf"):
        for pdf_45_135 in pathlib.Path(t_bin_45_135 / "plots").glob("*.pdf"):            
            if pdf_0_90.name == pdf_45_135.name and pdf_0_90.name in ["jp.pdf", "waves.pdf", "matrix.pdf"]:
                utils.big_print(f"0/90", 1.2)
                utils.display_pdf(str(pdf_0_90), 0)
                utils.big_print(f"45/135", 1.2)
                utils.display_pdf(str(pdf_45_135), 0)

### Background Discontinuity
Around the center of the spectrum, we see a discontinuity pop up where the background and other waves sharply change between two neighboring bins. To understand this, we'll observe the behavior of the 2 bins that surround the discontinuity.

#### Randomized Fits
To start, we'll want to check whether the best of the many randomized fits is a unique solution

In [ ]:
# determine the fit index associated with the neighboring bins in each t bin
t_to_mass_bin = {
    "t_0.1-0.2" : 1.40,
    "t_0.2-0.3" : 1.40,
    "t_0.3-0.5" : 1.42,
    "t_0.5-1.0" : 1.28,
}

t_to_indices = {}

for key, result in all_results_0_90.items():
    mass_bin = t_to_mass_bin[key]
    first_index = result.data_df[result.data_df["m_high"] == mass_bin]["fit_index"].values[0]
    t_to_indices[key] = [first_index + i for i in range(2)]    

In [ ]:
# verify the mass ranges
for t_bin, problem_indices in t_to_indices.items():
    print(
        all_results_0_90[t_bin].data_df[
            all_results_0_90[t_bin].data_df["fit_index"].isin(problem_indices)
        ][["fit_index", "m_low", "m_high"]]
    )

In [ ]:
for t_bin, problem_indices in t_to_indices.items():
    utils.big_print(f"t bin: {t_bin}", 2.0)
    for idx in problem_indices:
        # we know that D-wave errors are not correct right now, and that bootstrap data is not available for the moments
        # so we can suppress these warnings for now
        warnings.filterwarnings("ignore", category=UserWarning) 

        likelihood_cut = np.inf
        columns = [
            "p1p0S", 
            "p1p0D", 
            "p1mpP",
            all_results_0_90[t_bin].phase_difference_dict["p1p0S", "p1p0D"],
            all_results_0_90[t_bin].phase_difference_dict["p1p0S", "p1mmP"],
            all_results_0_90[t_bin].phase_difference_dict["p1p0S", "p1pmS"],
        ]

        mass_low, mass_high = all_results_0_90[t_bin].data_df[
            all_results_0_90[t_bin].data_df["fit_index"] == idx
        ][["m_low", "m_high"]].values[0]
        
        utils.big_print(f"mass: {mass_low:.2f}-{mass_high:.2f}", 1.5)

        utils.big_print(f"0/90", 1.2)
        all_results_0_90[t_bin].plot.randomized().randomized_summary(
            idx, likelihood_threshold=likelihood_cut, pwa_threshold=0.05, figsize=(15,10)
        )
        # all_results_0_90[t_bin].plot.randomized().randomized_summary(
        #     idx, likelihood_threshold=likelihood_cut, columns=columns
        # )
        plt.show()

        if t_bin == "t_0.3-0.5":
            # for this t bin and 45/135 orientation pair, the discontinuity is in the next bin
            idx += 1  
            mass_low, mass_high = all_results_45_135[t_bin].data_df[
                all_results_45_135[t_bin].data_df["fit_index"] == idx
            ][["m_low", "m_high"]].values[0]
            utils.big_print(f"mass: {mass_low:.2f}-{mass_high:.2f}", 1.5)

        utils.big_print(f"45/135", 1.2)
        all_results_45_135[t_bin].plot.randomized().randomized_summary(
            idx, likelihood_threshold=likelihood_cut, pwa_threshold=0.05, figsize=(15,10)
        )
        # all_results_45_135[t_bin].plot.randomized().randomized_summary(
        #     idx, likelihood_threshold=likelihood_cut, columns=columns
        # )
        plt.show()

warnings.resetwarnings()

#### Angular Distributions
Now lets look at the fits to the angles for any consistent change

In [ ]:
# use the same indices as above to grab the pdf pages
for t_bin, problem_indices in t_to_indices.items():
    utils.big_print(f"t bin: {t_bin}", 2.0)    

    dir_0_90 = pathlib.Path(f"{STUDY_DIR}/0-90/{t_bin}/plots")
    dir_45_135 = pathlib.Path(f"{STUDY_DIR}/45-135/{t_bin}/plots")
    
    for idx in problem_indices:
        mass_low, mass_high = all_results_0_90[t_bin].data_df[
            all_results_0_90[t_bin].data_df["fit_index"] == idx
        ][["m_low", "m_high"]].values[0]
        
        utils.big_print(f"mass: {mass_low:.2f}-{mass_high:.2f}", 1.5)

        utils.big_print(f"0/90", 1.2)
        utils.display_pdf(str(dir_0_90 / "combined_angles.pdf"), idx, 200)

        if t_bin == "t_0.3-0.5":
            # for this t bin and 45/135 orientation pair, the discontinuity is in the next bin
            idx += 1  
            mass_low, mass_high = all_results_45_135[t_bin].data_df[
                all_results_45_135[t_bin].data_df["fit_index"] == idx
            ][["m_low", "m_high"]].values[0]
            utils.big_print(f"mass: {mass_low:.2f}-{mass_high:.2f}", 1.5)

        utils.big_print(f"45/135", 1.2)        
        utils.display_pdf(str(dir_45_135 / "combined_angles.pdf"), idx, 200)

#### Alternative Solutions
The randomized fits seem to show some alternative solutions very far away in likelihood, with significant changes in the strongest D, S and P wave. Its possible that these solutions have angular distributions that match the neighboring bin's "best" solution. Lets use the first t bin of the 45/135 to investigate.

First we need to identify the randomized fit associated with these far likelihood values. We'll pick one since they seem to resolve to the same solution in the amplitudes and phases

In [ ]:
# we know the problem fit indices in this bin were 19 and 20 from above
t_bin = "t_0.1-0.2"
for idx in [19, 20]:
    print(
        all_results_45_135[t_bin].randomized_df[ # type: ignore
            all_results_0_90[t_bin].randomized_df["fit_index"] == idx # type: ignore
        ]["likelihood"].unique()
    )


In [ ]:
print(
    all_results_45_135[t_bin].randomized_df[
        (all_results_45_135[t_bin].randomized_df["fit_index"] == 19)
        & (all_results_45_135[t_bin].randomized_df["likelihood"] == -35314.4)
    ]["file"].to_list()[0]
)

print(
    all_results_45_135[t_bin].randomized_df[
        (all_results_45_135[t_bin].randomized_df["fit_index"] == 20)
        & (all_results_45_135[t_bin].randomized_df["likelihood"] == -33601.2)
    ]["file"].to_list()[0]
)

Now lets create pdfs of the angular distributions for these alternative fits

In [ ]:
%%bash

cd /lustre24/expphy/volatile/halld/home/kscheuer/ampToolsFits/omegapi/allPeriods/PARA_135_PERP_45/data/ver03/1m_1p_iso/recoil_pi_mass_1.4/t_0.10-0.20/mass_1.380-1.400/
cp ./rand/omegapi_2.fit ./alt.fit
vecps_plotter alt.fit
hadd ./vecps_plot_alt.root ./vecps_plot_omegapi_*.root
angle_plotter -f vecps_plot_alt.root -o $STUDY_DIR/45-135/alt_plots/bin1/ --gluex-style

cd /lustre24/expphy/volatile/halld/home/kscheuer/ampToolsFits/omegapi/allPeriods/PARA_135_PERP_45/data/ver03/1m_1p_iso/recoil_pi_mass_1.4/t_0.10-0.20/mass_1.400-1.420/
cp ./rand/omegapi_3.fit ./alt.fit
vecps_plotter alt.fit
hadd ./vecps_plot_alt.root ./vecps_plot_omegapi_*.root
angle_plotter -f vecps_plot_alt.root -o $STUDY_DIR/45-135/alt_plots/bin2/ --gluex-style


Now lets compare the alternative solution angular distributions to the "best" one in each bin

In [ ]:
# first, the bin before the discontinuity
t_bin = "t_0.1-0.2"
utils.display_pdf(f"{STUDY_DIR}/45-135/{t_bin}/plots/combined_angles.pdf", 19, 200)
utils.display_pdf(f"{STUDY_DIR}/45-135/alt_plots/bin1/angles.pdf", 0, 200)

In [ ]:
# now for the bin after the discontinuity
t_bin = "t_0.1-0.2"
utils.display_pdf(f"{STUDY_DIR}/45-135/{t_bin}/plots/combined_angles.pdf", 20, 200)
utils.display_pdf(f"{STUDY_DIR}/45-135/alt_plots/bin2/angles.pdf", 0, 200)

### b1 interference
Lets view how the stable the b1 vector interference is as a function of -t

In [ ]:
fig, axs = plt.subplots(
    2,
    4,
    sharex=True,    
    sharey="row",
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(15, 7),
    layout="constrained"
)
masses = result.plot.phase()._masses
bin_width = result.plot.phase()._bin_width
for i, t_bin in enumerate(all_results_0_90.keys()):
    result_0_90 = all_results_0_90[t_bin]
    result_45_135 = all_results_45_135[t_bin]    

    # ##### AMPLITUDES #####

    # --- 0/90 Orientations ---    
    amp1 = axs[0, i].errorbar(
        masses, 
        result_0_90.fit_df["p1p0S"],
        result_0_90.fit_df["p1p0S_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="tab:blue",
        label=rf"{utils.convert_amp_name("p1p0S")} (0/90)",
    )
    amp2 = axs[0, i].errorbar(
        masses, 
        result_0_90.fit_df["p1mpP"],
        result_0_90.fit_df["p1mpP_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="tab:orange",
        label=rf"{utils.convert_amp_name("p1mpP")} (0/90)",
    )

    # --- 45/135 Orientations ---
    amp1_45 = axs[0, i].errorbar(
        masses, 
        result_45_135.fit_df["p1p0S"],
        result_45_135.fit_df["p1p0S_err"],
        bin_width / 2,
        marker="",
        linestyle="-.",
        markersize=6,
        color="tab:blue",
        label=rf"{utils.convert_amp_name("p1p0S")} (45/135)",
    )
    amp2_45 = axs[0, i].errorbar(
        masses, 
        result_45_135.fit_df["p1mpP"],
        result_45_135.fit_df["p1mpP_err"],
        bin_width / 2,
        marker="",
        linestyle="-.",
        markersize=6,
        color="tab:orange",
        label=rf"{utils.convert_amp_name("p1mpP")} (45/135)",
    )

    # Fill between 0_90 and 45_135
    axs[0, i].fill_between(
        masses,
        result_0_90.fit_df["p1p0S"],
        result_45_135.fit_df["p1p0S"],
        color="tab:blue",
        alpha=0.2,
        step="mid"
    )
    axs[0, i].fill_between(
        masses,
        result_0_90.fit_df["p1mpP"],
        result_45_135.fit_df["p1mpP"],
        color="tab:orange",
        alpha=0.2,
        step="mid"
    )

    # ##### PHASES #####
    # --- 0/90 Orientations ---
    axs[1, i].errorbar(
        masses,
        result_0_90.fit_df[result_0_90.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        result_0_90.fit_df[f"{result_0_90.phase_difference_dict[('p1p0S', 'p1mpP')]}_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        color="black",
    )
    axs[1, i].errorbar(
        masses,
        -result_0_90.fit_df[result_0_90.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        result_0_90.fit_df[f"{result_0_90.phase_difference_dict[('p1p0S', 'p1mpP')]}_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        color="black",
    )    

    # --- 45/135 Orientations ---
    axs[1, i].errorbar(
        masses,
        result_45_135.fit_df[result_45_135.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        result_45_135.fit_df[f"{result_45_135.phase_difference_dict[('p1p0S', 'p1mpP')]}_err"],
        bin_width / 2,
        marker="",
        linestyle="-.",
        color="black",
    )
    axs[1, i].errorbar(
        masses,
        -result_45_135.fit_df[result_45_135.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        result_45_135.fit_df[f"{result_45_135.phase_difference_dict[('p1p0S', 'p1mpP')]}_err"],
        bin_width / 2,
        marker="",
        linestyle="-.",
        color="black",
    )

    # Fill between 0_90 and 45_135
    axs[1, i].fill_between(
        masses,
        result_0_90.fit_df[result_0_90.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        result_45_135.fit_df[result_45_135.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        color="black",        
        alpha=0.2,
        step="mid"
    )
    axs[1, i].fill_between(
        masses,
        -result_0_90.fit_df[result_0_90.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        -result_45_135.fit_df[result_45_135.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        color="black",        
        alpha=0.2,
        step="mid"
    )

    axs[0, i].set_ylim(bottom=0.0)
    low_t, high_t = t_bin.split('_')[1].split('-')
    axs[0, i].set_title(rf" {low_t} < $-t$ <  {high_t} GeV$^2$", pad=10)
    

    axs[1, i].set_yticks(np.linspace(-180, 180, 5))  # force to be in pi intervals
    axs[1, i].set_ylim([-180, 180])    
    axs[1, i].set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")

axs[0, 0].set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
axs[1, 0].set_ylabel(r"Phase Diff. ($^{\circ}$)", loc="center")

# make this a fig legend
axs[0, 3].legend(loc="upper right")

### D/S Ratio
Lets see how the d/s ratio and phase parameters move as a function of mass. The phase unfortunately for this wasn't preprocessed to be converted into degrees, so we'll apply the rad -> deg conversion here

In [ ]:
e852_ratio = 0.27
e852_phase = 10.54

fig, axs = plt.subplots(
    2,
    4,
    sharex=True,
    sharey="row",
    figsize=(15, 7),    
    layout="constrained",
)

for i, t_bin in enumerate(all_results_0_90.keys()):
    masses = all_results_0_90[t_bin].plot.phase()._masses
    bin_width = all_results_0_90[t_bin].plot.phase()._bin_width

    result_0_90 = all_results_0_90[t_bin]
    result_45_135 = all_results_45_135[t_bin]

    # ##### DS RATIO #####

    # --- 0/90 Orientations ---
    axs[0, i].axhline(y=e852_ratio, color="black", linestyle="--", alpha=0.7, label="E852")
    axs[0, i].errorbar(
        x=masses,
        xerr=bin_width / 2,
        y=result_0_90.fit_df["dsratio"],
        yerr=result_0_90.fit_df["dsratio_err"].abs(),
        marker="",
        linestyle="-",
        color="black",
        label="0/90",
    )

    # --- 45/135 Orientations ---
    axs[0, i].errorbar(
        x=masses,
        xerr=bin_width / 2,
        y=result_45_135.fit_df["dsratio"],
        yerr=result_45_135.fit_df["dsratio_err"].abs(),
        marker="",
        linestyle="-.",
        color="black",
        label="45/135",
    )

    # fill between 0_90 and 45_135
    axs[0, i].fill_between(
        masses,
        result_0_90.fit_df["dsratio"],
        result_45_135.fit_df["dsratio"],
        color="black",
        alpha=0.2,
        step="mid"
    )

    # ##### DS PHASE #####
    axs[1, i].axhline(y=-e852_phase, color="black", linestyle="--", alpha=0.7, label="E852")
    axs[1, i].axhline(y=e852_phase, color="black", linestyle="--", alpha=0.7)

    # --- 0/90 Orientations ---
    axs[1, i].errorbar(
        x=masses,
        xerr=bin_width / 2,
        y=result_0_90.fit_df["dphase"].abs().apply(np.rad2deg),
        yerr=result_0_90.fit_df["dphase_err"].abs().apply(np.rad2deg),
        marker="",
        linestyle="-",
        color="black",
        label="0/90",
    )
    axs[1, i].errorbar(
        x=masses,
        xerr=bin_width / 2,
        y=-result_0_90.fit_df["dphase"].abs().apply(np.rad2deg),
        yerr=result_0_90.fit_df["dphase_err"].abs().apply(np.rad2deg),
        marker="",
        linestyle="-",
        color="black",
    )

    # --- 45/135 Orientations ---
    axs[1, i].errorbar(
        x=masses,
        xerr=bin_width / 2,
        y=result_45_135.fit_df["dphase"].abs().apply(np.rad2deg),
        yerr=result_45_135.fit_df["dphase_err"].abs().apply(np.rad2deg),
        marker="",
        linestyle="-.",
        color="black",
        label="45/135",
    )
    axs[1, i].errorbar(
        x=masses,
        xerr=bin_width / 2,
        y=-result_45_135.fit_df["dphase"].abs().apply(np.rad2deg),
        yerr=result_45_135.fit_df["dphase_err"].abs().apply(np.rad2deg),
        marker="",
        linestyle="-.",
        color="black",
    )
    
    # fill between 0_90 and 45_135
    axs[1, i].fill_between(
        masses,
        result_0_90.fit_df["dphase"].abs().apply(np.rad2deg),
        result_45_135.fit_df["dphase"].abs().apply(np.rad2deg),
        color="black",
        alpha=0.2,
        step="mid"
    )
    axs[1, i].fill_between(
        masses,
        -result_0_90.fit_df["dphase"].abs().apply(np.rad2deg),
        -result_45_135.fit_df["dphase"].abs().apply(np.rad2deg),
        color="black",
        alpha=0.2,
        step="mid"
    )
    
    axs[0, i].set_ylim(0, 1)
    low_t, high_t = t_bin.split('_')[1].split('-')
    axs[0, i].set_title(rf" {low_t} < t < {high_t} ")

    axs[1, i].set_yticks(np.linspace(-180, 180, 5))
    axs[1, i].set_ylim(-180, 180)
    
axs[0, 0].set_ylabel("D/S Ratio", loc="top")    
axs[1, 0].set_ylabel("D-S Phase (°)", loc="top")
axs[1, -1].legend()


### Acceptance-Corrected Coherent Sums
To understand the production processes of the $J^P$ states, we can coherently sum over the $m$-projections and $\ell$ momenta to view their reflectivity dominance, and thus production method. We'll use the acceptance corrected values, as these lead later to a cross section.

In [ ]:
fig, axs = plt.subplots(
    2,
    4,    
    sharey="row",
    sharex=True,
    figsize=(15, 7),
    layout="constrained"
)

# Store handles and labels for the legend
handles = []
labels = []

for i, t_bin in enumerate(all_acc_corrected_results_0_90.keys()):
    result_0_90 = all_acc_corrected_results_0_90[t_bin]
    result_45_135 = all_acc_corrected_results_45_135[t_bin]  

    # ##### 1P COHERENT SUM #####  

    # --- 0/90 Orientations ---    
    h1 = axs[0, i].errorbar(
        masses, 
        result_0_90.fit_df["p1p"],
        result_0_90.fit_df["p1p_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="darkred",
        label=rf"$1^+$ (Natural)",
    )
    h2 = axs[0, i].errorbar(
        masses, 
        result_0_90.fit_df["p1m"],
        result_0_90.fit_df["p1m_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="lightcoral",
        label=rf"$1^-$ (Natural)",
    )

    # --- 45/135 Orientations ---
    h3 = axs[0, i].errorbar(
        masses, 
        result_45_135.fit_df["p1p"],
        result_45_135.fit_df["p1p_err"],
        bin_width / 2,
        marker="",
        linestyle="-.",
        markersize=6,
        color="darkred",
    )
    h4 = axs[0, i].errorbar(
        masses, 
        result_45_135.fit_df["p1m"],
        result_45_135.fit_df["p1m_err"],
        bin_width / 2,
        marker="",
        linestyle="-.",
        markersize=6,
        color="lightcoral",        
    )

    # Fill between 0_90 and 45_135
    axs[0, i].fill_between(
        masses,
        result_0_90.fit_df["p1p"],
        result_45_135.fit_df["p1p"],
        color="darkred",
        alpha=0.2,
        step="mid"
    )
    axs[0, i].fill_between(
        masses,
        result_0_90.fit_df["p1m"],
        result_45_135.fit_df["p1m"],
        color="lightcoral",
        alpha=0.2,
        step="mid"
    )

    # ##### 1M COHERENT SUM #####

    # --- 0/90 Orientations ---    
    h5 = axs[1, i].errorbar(
        masses, 
        result_0_90.fit_df["m1p"],
        result_0_90.fit_df["m1p_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="darkblue",
        label=rf"$1^+$ (Unnatural)",
    )
    h6 = axs[1, i].errorbar(
        masses, 
        result_0_90.fit_df["m1m"],
        result_0_90.fit_df["m1m_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="lightblue",
        label=rf"$1^-$ (Unnatural)",
    )

    # --- 45/135 Orientations ---
    h7 = axs[1, i].errorbar(
        masses, 
        result_45_135.fit_df["m1p"],
        result_45_135.fit_df["m1p_err"],
        bin_width / 2,
        marker="",
        linestyle="-.",
        markersize=6,
        color="darkblue",
    )
    h8 = axs[1, i].errorbar(
        masses, 
        result_45_135.fit_df["m1m"],
        result_45_135.fit_df["m1m_err"],
        bin_width / 2,
        marker="",
        linestyle="-.",
        markersize=6,
        color="lightblue",        
    )

    # Fill between 0_90 and 45_135
    axs[1, i].fill_between(
        masses,
        result_0_90.fit_df["m1p"],
        result_45_135.fit_df["m1p"],
        color="darkblue",
        alpha=0.2,
        step="mid"
    )
    axs[1, i].fill_between(
        masses,
        result_0_90.fit_df["m1m"],
        result_45_135.fit_df["m1m"],
        color="lightblue",
        alpha=0.2,
        step="mid"
    )

    # Collect legend handles and labels only from the first subplot to avoid duplicates
    if i == 0:
        handles.extend([h1, h2, h5, h6,])
        labels.extend([h.get_label() for h in [h1, h2, h5, h6]])

    low_t, high_t = t_bin.split('_')[1].split('-')
    axs[0, i].set_title(rf" {low_t} < $-t$ <  {high_t} GeV$^2$", pad=10, fontsize=16)
    axs[0, i].set_ylim(bottom=0.0)
    axs[1, i].set_ylim(bottom=0.0)
    axs[1, i].set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")

axs[0, 0].set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
axs[1, 0].set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
axs[1, 0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))

# Create a single legend for the entire figure positioned on the right
fig.legend(handles, labels, loc='center right', bbox_to_anchor=(1.2, 0.5))

### Projected Moments

#### All Moments

In [ ]:
# Identify moment columns (assuming they start with 'moment_' or similar pattern)
moment_cols = [col for col in all_results_0_90[next(iter(all_results_0_90))].proj_moments_df.columns if "H" in col] # type:ignore

num_moments = len(moment_cols)
ncols = 4
nrows = int(np.ceil(num_moments / ncols))

for t_bin in all_results_0_90.keys():

    fig, axs = plt.subplots(
        nrows, ncols,
        figsize=(4 * ncols, 3 * nrows),
        sharex=True,
        layout="constrained"
    )

    for i, moment in enumerate(moment_cols):
        ax = axs.flat[i]
        
        masses = all_results_0_90[t_bin].plot.phase()._masses
        bin_width = all_results_0_90[t_bin].plot.phase()._bin_width
    
        proj_moments_0_90 = all_results_0_90[t_bin].proj_moments_df
        proj_moments_45_135 = all_results_45_135[t_bin].proj_moments_df

        assert proj_moments_0_90 is not None
        assert proj_moments_45_135 is not None

        # 0/90 orientation
        ax.plot(
            masses,
            proj_moments_0_90[moment],                        
            marker="",
            linestyle="-",
            color="black",            
            label="0/90"
        )
        # 45/135 orientation
        ax.plot(
            masses,
            proj_moments_45_135[moment],
            marker="",
            linestyle="-.",
            color="black",            
            label="45/135"
        )        
        # Fill between 0_90 and 45_135
        ax.fill_between(
            masses,
            proj_moments_0_90[moment],
            proj_moments_45_135[moment],
            color="black",
            alpha=0.2,
            step="mid"
        )

        ax.set_title(moment)        
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        # Only set xlabel for last row
        if i // ncols == nrows - 1:
            ax.set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")

    # Hide unused subplots if any
    for j in range(num_moments, nrows * ncols):
        fig.delaxes(axs.flat[j])

    

    t_low, t_high = t_bin.split('_')[1].split('-')
    fig.suptitle(rf"Projected Moments: {t_low} < -t < {t_high} GeV$^2$", fontsize=16)
    plt.show()

#### Significant Moments

In [ ]:
moments = [c for c in all_results_0_90[next(iter(all_results_0_90))].proj_moments_df.columns if c.startswith("H")] # type: ignore
significant_moments = []
for m in moments:
    for t_bin, result in all_results_0_90.items():
        assert result.proj_moments_df is not None
        H0_0000 = result.proj_moments_df["H0_0000"].values[0] 
        if (result.proj_moments_df[m] / H0_0000).abs().mean() > 0.05:
            significant_moments.append(m)
            break

In [ ]:
h0_moments = [m for m in significant_moments if m.startswith("H0")]
h1_moments = [m for m in significant_moments if m.startswith("H1")]
h2_moments = [m for m in significant_moments if m.startswith("H2")]

In [ ]:
def plot_moments_grid(moment_list, title_prefix):
    nrows = len(moment_list)
    ncols = 4
    fig, axs = plt.subplots(
        nrows, ncols,
        figsize=(4 * ncols, 2.5 * nrows),
        sharex=True,
        sharey="row",
        layout="constrained"
    )

    for col_idx, t_bin in enumerate(all_results_0_90.keys()):
        masses = all_results_0_90[t_bin].plot.phase()._masses        
        proj_moments_0_90 = all_results_0_90[t_bin].proj_moments_df
        proj_moments_45_135 = all_results_45_135[t_bin].proj_moments_df

        assert proj_moments_0_90 is not None
        assert proj_moments_45_135 is not None

        for row_idx, moment in enumerate(moment_list):
            ax = axs[row_idx, col_idx] if nrows > 1 else axs[col_idx]
            ax.plot(
                masses,
                proj_moments_0_90[moment],
                marker="",
                linestyle="-",
                color="black",
                label="0/90" if col_idx == 0 else None
            )
            ax.plot(
                masses,
                proj_moments_45_135[moment],
                marker="",
                linestyle="-.",
                color="black",
                label="45/135" if col_idx == 0 else None
            )
            ax.fill_between(
                masses,
                proj_moments_0_90[moment],
                proj_moments_45_135[moment],
                color="black",
                alpha=0.2,
                step="mid"
            )
            if col_idx == 0:
                ax.set_ylabel(utils.convert_moment_name(moment))
            if row_idx == nrows - 1:
                ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", loc="right")
            if row_idx == 0:
                low_t, high_t = t_bin.split('_')[1].split('-')
                ax.set_title(rf"{low_t} < -t < {high_t} GeV$^2$", fontsize=12)
            ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
            ax.grid(True)

    handles, labels = [], []
    for ax in axs.flat if nrows > 1 else axs:
        for h, l in zip(*ax.get_legend_handles_labels()):
            if l not in labels:
                handles.append(h)
                labels.append(l)
    # fig.legend(handles, labels, loc='center right', bbox_to_anchor=(1.15, 0.5))
    fig.suptitle(f"{title_prefix} Projected Moments", fontsize=16)
    plt.show()

plot_moments_grid(h0_moments, "H0")
plot_moments_grid(h1_moments, "H1")
plot_moments_grid(h2_moments, "H2")